In [1]:
import pandas as pd

# Path to the CSV file in the ESC-50 dataset
Path = '/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/ESC-50- UTILITY/esc-50/ESC-50-master/'
csv_path = Path + 'meta/esc50.csv'

# Load the CSV into a DataFrame
df = pd.read_csv(csv_path)
audio_path_prefix = Path+'/audio/'


df = df[['filename', 'category']]

print(df.head(10))


            filename         category
0   1-100032-A-0.wav              dog
1  1-100038-A-14.wav   chirping_birds
2  1-100210-A-36.wav   vacuum_cleaner
3  1-100210-B-36.wav   vacuum_cleaner
4  1-101296-A-19.wav     thunderstorm
5  1-101296-B-19.wav     thunderstorm
6  1-101336-A-30.wav  door_wood_knock
7  1-101404-A-34.wav      can_opening
8   1-103298-A-9.wav             crow
9  1-103995-A-30.wav  door_wood_knock


In [2]:
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [3]:
df = pd.read_csv(csv_path)
df['category'].unique()
# df['target'].unique()
# df[['target', 'category']]

array(['dog', 'chirping_birds', 'vacuum_cleaner', 'thunderstorm',
       'door_wood_knock', 'can_opening', 'crow', 'clapping', 'fireworks',
       'chainsaw', 'airplane', 'mouse_click', 'pouring_water', 'train',
       'sheep', 'water_drops', 'church_bells', 'clock_alarm',
       'keyboard_typing', 'wind', 'footsteps', 'frog', 'cow',
       'brushing_teeth', 'car_horn', 'crackling_fire', 'helicopter',
       'drinking_sipping', 'rain', 'insects', 'laughing', 'hen', 'engine',
       'breathing', 'crying_baby', 'hand_saw', 'coughing',
       'glass_breaking', 'snoring', 'toilet_flush', 'pig',
       'washing_machine', 'clock_tick', 'sneezing', 'rooster',
       'sea_waves', 'siren', 'cat', 'door_wood_creaks', 'crickets'],
      dtype=object)

In [4]:
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset metadata
csv_path = Path+'/meta/esc50.csv'
df = pd.read_csv(csv_path)


<h1> Flatten Mel 

In [1]:
# Audio Processing Classes
class AudioFeatureExtractor:
    def process_audio(self, audio, sr):
        raise NotImplementedError

class RawAudioExtractor(AudioFeatureExtractor):
    def process_audio(self, audio, sr):
        return audio
    

In [2]:
import os
import json
import torch
import numpy as np

from init_config import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class LogisticRegressionClassifier(torch.nn.Module):
    def __init__(self, feature_dim=129):
        super(LogisticRegressionClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(feature_dim, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, xx):
        return self.sigmoid(self.linear1(torch.nn.functional.normalize(xx, p=1.0, dim=1)))


def load_kirigami_model():
    # Phoneme Model!
    my_phoneme_filter_model = LogisticRegressionClassifier(feature_dim=129)
    my_phoneme_filter_model.load_state_dict(
    torch.load(lr_phoneme_checkpoint_path, map_location=torch.device('cpu')))
    my_phoneme_filter_model.eval()

    return my_phoneme_filter_model


def load_background_filter_model():
    # Background Model!
    my_background_filter_model = LogisticRegressionClassifier(feature_dim=129)
    my_background_filter_model.load_state_dict(torch.load(bg_lr_checkpoint_path, map_location=torch.device('cpu')))
    my_background_filter_model.eval()

    return my_background_filter_model


def kirigami_filter_torch(s_full, threshold=0.5):
    lr_phoneme_filter_model = LogisticRegressionClassifier(feature_dim=129)
    # load the model if in kirigami_filters directory
    if os.path.exists("./kirigami_filters/scipy_phoneme_filter.ckpt"):
        lr_phoneme_filter_model.load_state_dict(
            torch.load("./kirigami_filters/scipy_phoneme_filter.ckpt", map_location=device))
    else:
        raise FileNotFoundError("Phoneme filter model not found")

    lr_phoneme_filter_model.eval()
    pred = (lr_phoneme_filter_model.forward(torch.Tensor(s_full)) >= threshold).long().numpy()
    masked = (1 - pred) * s_full
    return masked


def kirigami_filter(stft):
    output_sp = np.zeros_like(stft)
    for i, fft in enumerate(stft):

        sum = np.sum(fft)

        product = 0
        for iw, (vv, ww) in enumerate(zip(fft, weight)):
            product = product + vv * weight[iw]
        product = product / sum
        product = product + bias

        z = 1 / (1 + np.exp(-product))
        # print("LR filter probability", i, z)
        if z < LR_THRESHOLD:
            # add the value
            output_sp[i] = stft[i]
    return output_sp


def kirigami_filter_reverse_fft(stft, stft_original):
    output_sp = np.zeros_like(stft)
    for i, fft in enumerate(stft):
        sum = np.sum(fft)
        product = 0
        for iw, (vv, ww) in enumerate(zip(fft, weight)):
            product = product + vv * weight[iw]
        product = product / sum
        product = product + bias
        z = 1 / (1 + np.exp(-product))
        # print("LR filter probability", i, z)
        if z < LR_THRESHOLD:
            # add the value
            # output_sp[i] = stft[i]
            output_sp[i] = stft_original[i]
    return output_sp


def background_detection_filter(stft):
    output_sp = np.zeros_like(stft)
    for i, fft in enumerate(stft):
        sum = np.sum(fft)
        product = 0
        for iw, (vv, ww) in enumerate(zip(fft, weight_background)):
            product = product + vv * weight_background[iw]
        product = product // sum
        product = product + bias_background
        z = 1 / (1 + np.exp(-product))
        # print("Background probability: ", i, z)
        if z < BACKGROUND_LR_THRESHOLD:  # lower than threshold not background.
            # add the value
            output_sp[i] = stft[i]
    return output_sp


import os
import librosa
import numpy as np
import torch
import soundfile as sf
import matplotlib.pyplot as plt

# Load Kirigami models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_kirigami_model():
    """Load the phoneme filter model."""
    model = LogisticRegressionClassifier(feature_dim=129)
    model.load_state_dict(torch.load("kirigami_filters/model_checkpoints/scipy_phoneme_filter.ckpt", map_location=device))
    model.eval()
    return model

def load_background_filter_model():
    """Load the background noise filter model."""
    model = LogisticRegressionClassifier(feature_dim=129)
    model.load_state_dict(torch.load("kirigami_filters/model_checkpoints/noisy_background_scipy_detector.ckpt", map_location=device))
    model.eval()
    return model

# Load models
phoneme_filter_model = load_kirigami_model()
background_filter_model = load_background_filter_model()

# Extract model weights
weight_phoneme = phoneme_filter_model.linear1.weight.data[0].numpy()
bias_phoneme = phoneme_filter_model.linear1.bias.data[0].numpy()
weight_background = background_filter_model.linear1.weight.data[0].numpy()
bias_background = background_filter_model.linear1.bias.data[0].numpy()


class KirigamiExtractor(AudioFeatureExtractor):
    def apply_kirigami_filter(self, stft, weight, bias, threshold=0.5):
        """Apply Kirigami logistic regression filter on STFT features (ensuring 129 features per frame)."""
        output_stft = np.zeros_like(stft)

        for i, frame in enumerate(stft):
            frame = frame[:129]  # Ensure exactly 129 dimensions

            sum_val = np.sum(frame) + 1e-6  # Avoid division by zero
            product = np.dot(frame, weight) / sum_val + bias
            prob = 1 / (1 + np.exp(-product))  # Sigmoid activation

            if prob < threshold:  # If probability is low, keep frame
                output_stft[i, :129] = frame  # Apply only to the valid region

        return output_stft

    # Process Audio File
    def process_audio(self, audio, sr=16000, threshold=0.5):
        """Process an audio file through Kirigami models and save the filtered output."""
        # Load audio
        # audio, sr = librosa.load(input_audio_path, sr=16000)

        # Compute STFT (Ensure output has 129 feature bins)
        stft = np.abs(librosa.stft(audio, n_fft=256, hop_length=128))[:129, :].T  # Transpose for correct shape

        # Apply Kirigami phoneme & background filters
        filtered_stft_phoneme = self.apply_kirigami_filter(stft, weight_phoneme, bias_phoneme, threshold)
        # filtered_stft_background = self.apply_kirigami_filter(filtered_stft_phoneme, weight_background, bias_background, threshold)

        # Convert back to audio using inverse STFT
        filtered_audio = librosa.istft(filtered_stft_phoneme.T, hop_length=128)

        return filtered_audio





In [3]:
import os
import numpy as np
import pandas as pd
import librosa
from scipy import signal
from sklearn.decomposition import PCA

import os
import numpy as np
import librosa
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score





# Set the correct absolute path
base_path = "/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/ESC-50- UTILITY/esc-50/ESC-50-master"

# Load the dataset metadata
csv_path = os.path.join(base_path, "meta", "esc50.csv")

# Ensure dataset exists
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"Error: The file '{csv_path}' does not exist. Please check the path.")

df = pd.read_csv(csv_path)

# Define the audio file path
audio_path_prefix = os.path.join(base_path, "audio")

# Ensure the audio directory exists
if not os.path.exists(audio_path_prefix):
    raise FileNotFoundError(f"Error: The audio directory '{audio_path_prefix}' does not exist.")
    
    



# class KirigamiExtractor(AudioFeatureExtractor):
#     def __init__(self, model_path="/work/pi_shenoy_umass_edu/sgomasta_umass_edu/Batch Files/Privacy Peserving Audio/Common Voice/bhawana/kirigami3.pkl", sr=16000, n_fft=256, hop_length=128, window_size=1.0, stride=0.5):
#         """Initialize Kirigami for filtering after loading the trained model."""
#         self.sr = sr
#         self.n_fft = n_fft
#         self.hop_length = hop_length
#         self.window_size = int(window_size * sr)
#         self.stride = int(stride * sr)
#         self.model_path = model_path
#         with open(self.model_path, "rb") as f:
#                 self.model = pickle.load(f)
#     def process_audio(self, audio, sr):
#         """Apply the speech filter window-by-window."""
#         num_samples = len(audio)
#         filtered_audio = np.zeros_like(audio)

#         for start in range(0, num_samples - self.window_size, self.stride):
#             window = audio[start : start + self.window_size]
#             stft = np.abs(librosa.stft(window, n_fft=self.n_fft, hop_length=self.hop_length))
#             norm = np.linalg.norm(stft, ord=1, axis=0, keepdims=True)
#             norm[norm == 0] = 1
#             stft = stft / norm
#             features = np.nan_to_num(stft.T)
#             prediction = self.model.predict(features.reshape(1, -1))

#             if prediction < 0.5:
#                 filtered_audio[start : start + self.window_size] = window  # Keep non-speech

#         return filtered_audio


    


class SyntheticSensorExtractor(AudioFeatureExtractor):
    def process_audio(self, audio, sr):
        return signal.resample(audio, len(audio) // 2)

class PrivacyMicExtractor(AudioFeatureExtractor):
    def process_audio(self, audio, sr):
        S = librosa.stft(audio, n_fft=256, hop_length=128)
        frequencies = librosa.fft_frequencies(sr=sr, n_fft=256)
        S_filtered = np.where(frequencies[:, None] <= 300, S, 0)
        return librosa.istft(S_filtered, hop_length=128)

class CoughSenseExtractor(AudioFeatureExtractor):
    def process_audio(self, audio, sr):
        n_fft = int(0.15 * sr)
        hop_length = n_fft // 2
        S = np.abs(librosa.stft(audio.astype(np.float64), n_fft=n_fft, hop_length=hop_length))
        pca = PCA(n_components=10)
        S_reduced = pca.fit_transform(S.T)
        S_reconstructed = pca.inverse_transform(S_reduced).T
        return librosa.istft(S_reconstructed, hop_length=hop_length)

class SamosaExtractor(AudioFeatureExtractor):
    def process_audio(self, audio, sr):
        subsampled_audio = signal.resample(audio, len(audio) // 2)
        return signal.resample(subsampled_audio, len(audio))

# Feature Extraction Function (Flattened Mel-Spectrogram)
def extract_melspectrogram(audio, sr, label, window_size=0.5, hop_size=0.3):
    """Extracts Mel-Spectrogram features using a sliding window approach, limited to 10 windows per file."""
    window_length = int(window_size * sr)
    hop_length = int(hop_size * sr)

    all_features = []
    window_count = 0

    for start in range(0, len(audio) - window_length + 1, hop_length):

        window_segment = audio[start:start + window_length]
        # if len(window_segment) != window_length:
        #     continue  

        # Compute Mel-Spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y=window_segment, sr=sr)

        # Flatten the spectrogram to a 1D feature vector
        mel_spectrogram_flattened = mel_spectrogram.mean(axis=1)

        # Create dictionary of features
        features = {f"mel_{i}": mel_spectrogram_flattened[i] for i in range(len(mel_spectrogram_flattened))}
        features['Label'] = label
        features['Window Index'] = window_count  # Track window index

        all_features.append(features)
        window_count += 1  

    return pd.DataFrame(all_features)

# Initialize Extractors
extractors = {  "kirigami":KirigamiExtractor(),
    #"RawAudio": RawAudioExtractor(),
    #"SyntheticSensor": SyntheticSensorExtractor(),
    #"PrivacyMic": PrivacyMicExtractor(),
   # "CoughSense": CoughSenseExtractor(),
    #"Samosa": SamosaExtractor()
    # "kirigami":KirigamiExtractor()
}

# Extract features for each filtering technique
for extractor_name, extractor in extractors.items():
    print(f"Processing {extractor_name} filtering...")

    features_list = pd.DataFrame()
    
    for _, row in df.iterrows():
        file_path = os.path.join(audio_path_prefix, row['filename'])

        if not os.path.exists(file_path):
            print(f"Warning: Skipping missing file {file_path}")
            continue

        # Load raw audio
        y, sr = librosa.load(file_path, sr=None)

        # Process with the chosen filtering method
        processed_audio = extractor.process_audio(y, sr)

        # Extract features
        features = extract_melspectrogram(processed_audio, sr, row['target'])
        features_list = features_list._append(features, ignore_index=True)

    # Save the extracted features
    output_file = f"flattened_mel_spectrogram_{extractor_name}.csv"
    features_list.to_csv(output_file, index=False)
    print(f"Feature extraction completed for {extractor_name}. Saved to {output_file}")

print("All feature extraction processes are complete.")


Processing kirigami filtering...
Feature extraction completed for kirigami. Saved to flattened_mel_spectrogram_kirigami.csv
All feature extraction processes are complete.


In [4]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# List of all processed feature CSVs
csv_files = [
     "flattened_mel_spectrogram_RawAudio.csv",
    "flattened_mel_spectrogram_kirigami.csv",
    "flattened_mel_spectrogram_PrivacyMic.csv",
    "flattened_mel_spectrogram_Samosa.csv",
    "flattened_mel_spectrogram_SyntheticSensor.csv"
   
    
]

# Dictionary to store results
metrics_results = {}

# Iterate over each CSV file and train a model
for selected_csv in csv_files:
    print(f"\n🔹 Processing dataset: {selected_csv}")

    # Load dataset
    df = pd.read_csv(selected_csv)

    # Ensure Mel-Spectrogram columns exist
    feature_cols = [col for col in df.columns if col.startswith("mel_")]
    if not feature_cols:
        raise ValueError(f"Error: No 'mel_' feature columns found in {selected_csv}.")

    # Remove rows where all Mel-Spectrogram features are zero
    df_filtered = df.loc[~(df[feature_cols] == 0).all(axis=1)]

    # Check if the dataset is empty after filtering
    if df_filtered.empty:
        raise ValueError(f"Error: After removing zero-only feature rows, {selected_csv} has no data left.")

    X = df_filtered[feature_cols].values  # Feature matrix
    y = df_filtered["Label"].values  # Labels

    # Split into train and test sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Initialize and train the Random Forest classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Compute Precision, Recall, and F1-score for each class
    report = classification_report(y_test, y_pred, digits=4, output_dict=True)

    # Store results in dictionary
    metrics_results[selected_csv] = {
        "Precision": report["macro avg"]["precision"],
        "Recall": report["macro avg"]["recall"],
        "F1-score": report["macro avg"]["f1-score"],
        "Accuracy": accuracy_score(y_test, y_pred)
    }

    # Print classification report
    print(f"✅ Model trained on {selected_csv} features.")
    print(f"🎯 Accuracy: {metrics_results[selected_csv]['Accuracy']:.4f}\n")
    print(f"📊 Classification Report:\n{classification_report(y_test, y_pred, digits=4)}")

# Convert metrics to a DataFrame for better visualization
df_metrics = pd.DataFrame.from_dict(metrics_results, orient="index")

# Save extracted metrics to CSV
output_csv = "classification_metrics_summary.csv"
df_metrics.to_csv(output_csv, index=True)

# Display the extracted results
print(f"\n📄 Metrics extracted and saved to {output_csv}")



🔹 Processing dataset: flattened_mel_spectrogram_RawAudio.csv
✅ Model trained on flattened_mel_spectrogram_RawAudio.csv features.
🎯 Accuracy: 0.8420

📊 Classification Report:
              precision    recall  f1-score   support

           0     0.7830    0.7477    0.7650       111
           1     0.7654    0.6966    0.7294        89
           2     0.8333    0.7895    0.8108       114
           3     0.9266    0.8783    0.9018       115
           4     0.9248    0.9609    0.9425       128
           5     0.7475    0.6789    0.7115       109
           6     0.8106    0.8843    0.8458       121
           7     0.8429    0.9593    0.8973       123
           8     0.8696    0.9449    0.9057       127
           9     0.9344    0.9661    0.9500       118
          10     0.9014    1.0000    0.9481       128
          11     0.8176    0.9453    0.8768       128
          12     0.8741    0.9440    0.9077       125
          13     0.9624    1.0000    0.9808       128
          14  